# Generate frames from video

In [3]:
import numpy as np
import cv2
from IPython.display import Video, HTML
from pathlib import Path

local_path = Path('.')

## Import video

In [4]:
video_path = local_path / 'NightSky.mp4'
video_cap = cv2.VideoCapture(str(video_path))
video_width  = video_cap.get(cv2.CAP_PROP_FRAME_WIDTH)
video_height = video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
video_fps    = video_cap.get(cv2.CAP_PROP_FPS)
video_name   = video_path.stem
video_nbr_frame = int(video_cap.get(cv2.CAP_PROP_FRAME_COUNT))

HTML("""
    <video width="858" height="480" controls>
        <source src="{}" type="video/mp4">
    </video>
    """.format(video_path))

## Parameters and folder creation

In [5]:
dataset_path = local_path / (video_name + '_dataset')
extraction_fps = 3
extraction_resolution = 1024

if (dataset_path).is_dir():
    if len([x for x in dataset_path.glob('**/*')]) > 0:
        print('\N{Heavy Exclamation Mark Symbol} Dataset already created \N{Heavy Exclamation Mark Symbol}')
        print('Delete current dataset folder ({}) to regenerate images.'.format(dataset_path))
else:
    %mkdir "{dataset_path}"

❗ Dataset already created ❗
Delete current dataset folder (NightSky_dataset) to regenerate images.


In [4]:
frame_counter_out = 0

for counter in range(video_nbr_frame):
    if (video_cap.isOpened()):
        ret, frame = video_cap.read()

        if counter % int(video_fps / extraction_fps) == 0:
            frame_counter_out += 1
            frame_squared = cv2.resize(frame, (extraction_resolution,extraction_resolution), interpolation = cv2.INTER_AREA)
            cv2.imshow('{} frame extraction'.format(video_name),frame_squared)
            cv2.imwrite(str(dataset_path / (video_name + '_{}.jpg'.format(frame_counter_out))), frame_squared)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    else:
        break

In [5]:
video_cap.release()
cv2.destroyAllWindows()

## Check dataset

In [8]:
for path in dataset_path.glob('**/*.jpg'):
    path_str = str(path)
    img = cv2.imread(path_str)
    dimensions = img.shape
    if dimensions != (1024,1024,3):
        print('Error')